In [1]:
#Load dataset
import pandas as pd
df = pd.read_csv("clean_occupation_texts.csv")

In [2]:
df

,Element Name,occupation_text
0,Chief Executives,Generated description:\nIndividuals in this ro...
1,Chief Sustainability Officers,Generated description:\nIndividuals in this ro...
2,General and Operations Managers,Generated description:\nIndividuals in this ro...
3,Advertising and Promotions Managers,Generated description:\nIndividuals in this ro...
4,Marketing Managers,Generated description:\nProfessionals in this ...
...,...,...
889,Gas Compressor and Gas Pumping Station Operators,Generated description:\nThese professionals ov...
890,"Pump Operators, Except Wellhead Pumpers",Generated description:\nIndividuals in this ro...
891,Wellhead Pumpers,Generated description:\nIndividuals in this ro...
892,Refuse and Recyclable Material Collectors,Generated description:\nIndividuals in this ro...


In [3]:
df.shape

(894, 2)

In [4]:
pip install groq


Note: you may need to restart the kernel to use updated packages.


### API Authentication

In the following we setting up the API Configuartion. We will use Gemini for processing the text data.

In [5]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional, Dict
from datetime import date
import time
from tqdm import tqdm
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# IMPORTANT: put your Groq key in .env as GROQ_API_KEY=sk-...
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise RuntimeError("Set GROQ_API_KEY in your .env or environment variables.")


### Test Connection

Let's do a quick check to ensure we can connect to the API.

In [6]:
# Create Groq client
client = Groq(api_key=api_key)

# Quick sanity check
test_resp = client.chat.completions.create(
    model="llama-3.1-8b-instant",  # good default Groq model
    messages=[
        {"role": "user", "content": "Say hi in one short sentence."}
    ],
    temperature=0,
)

print("✅ Groq client works. Model reply:", test_resp.choices[0].message.content)

✅ Groq client works. Model reply: Hello.


Schema definition:

In [7]:
from pydantic import BaseModel, Field
from typing import List

class SkillList(BaseModel):
    skills: List[str] = Field(
        description=(
            "A list of 10–14 general, reusable skills (2–4 word noun phrases) "
            "needed to perform the occupation."
        )
    )


Prompt:

In [8]:
skill_extraction_instructions = """
You are an expert in occupations and skills extraction.

Your task is to return ONLY a JSON object with a single key "skills".
Do NOT include explanations, commentary, or any text outside the JSON.

Task:
Using the occupation title and full occupation text provided below
(which may include descriptions, tasks, and skills with importance scores),
extract a list of concrete, generalizable skills a worker needs to perform this job.

WHAT COUNTS AS A SKILL
- a short noun phrase (2–4 words)
- a generalizable capability (not a specific action)
- something reusable across occupations

WHAT DOES NOT COUNT AS A SKILL
- micro-tasks ("shampooing hair", "towel drying hair")
- verb phrases ("coordinating schedules", "teaching procedures")
- job-specific motions ("cutting grass", "loading packages")
- duplicate variants ("hair washing", "cleansing hair")

HOW TO GENERALIZE CORRECTLY
Convert actions → capability nouns:
- "shampooing hair" → "hair cleansing techniques"
- "analyzing financial statements" → "financial analysis"
- "teaching customers how to use products" → "customer education"

IMPORTANCE SCORES
When selecting skills, give more weight to skills with higher importance scores,
but do NOT mention the scores in your output.

SKILL FORMAT CONSTRAINTS
- Each skill must be plain text without quotation marks inside it.
- Do NOT use commas or semicolons inside a skill.
- Use only letters, numbers, spaces, and hyphens in skill names.

FINAL OUTPUT RULES (VERY IMPORTANT)
- Return ONLY a JSON object.
- Return between 12 and 14 total skills. Never return more than 14 skills.
- The top-level object MUST have exactly one key: "skills".
- "skills" MUST be a JSON array of strings.
- Do NOT wrap the JSON in ``` or ```json.
- Do NOT add any text before or after the JSON.
- Do NOT add explanations or notes.

Example of the required structure (do NOT copy these exact values):
{"skills": ["skill one", "skill two", "skill three"]}
""".strip()


Helper Function to make Groq outputs safe to parse

In [9]:
import json

def clean_json_text(text: str) -> str:
    """
    Takes the raw LLM response.
    If it's wrapped in ```json ... ``` or ``` ... ``` fences,
    remove those and return just the inner JSON string.
    """
    if not isinstance(text, str):
        return ""

    s = text.strip()

    # Strip triple backticks if they exist
    if s.startswith("```"):
        first_newline = s.find("\n")
        if first_newline != -1:
            s = s[first_newline + 1:]

        last_fence = s.rfind("```")
        if last_fence != -1:
            s = s[:last_fence]

        s = s.strip()

    return s


Try with 10 

In [10]:
# 10 samples
df_few = df.head(10).copy()   # change to df.sample(10, random_state=1) if you want random

print(df_few.head())
print(df_few.shape)


                          Element Name  \
0                     Chief Executives   
1        Chief Sustainability Officers   
2      General and Operations Managers   
3  Advertising and Promotions Managers   
4                   Marketing Managers   

                                     occupation_text  
0  Generated description:\nIndividuals in this ro...  
1  Generated description:\nIndividuals in this ro...  
2  Generated description:\nIndividuals in this ro...  
3  Generated description:\nIndividuals in this ro...  
4  Generated description:\nProfessionals in this ...  
(10, 2)


In [11]:
# --- Use the small test dataframe (10 rows) ---
df_test = df_few.copy()

# --- Add output column if missing ---
if "llm_skills" not in df_test.columns:
    df_test["llm_skills"] = None

# --- Rate limit config ---
rows_per_minute = 20
delay = 60 / rows_per_minute   # 3 seconds per request

for i in tqdm(range(len(df_test)), desc="Extracting skills", unit="job"):
    row = df_test.iloc[i]

    occupation = str(row.get("Element Name", "") or "").strip()
    occupation_text = str(row.get("occupation_text", "") or "").strip()

    if not occupation_text:
        df_test.loc[i, "llm_skills"] = None
        continue

    # ----------------------------
    # SYSTEM + USER messages
    # JSON mode ON
    # ----------------------------
    try:
        completion = client.with_options(timeout=60.0).chat.completions.create(
            model="llama-3.1-8b-instant",
            temperature=0,
            response_format={"type": "json_object"},   # 🔴 force valid JSON
            messages=[
                {
                    "role": "system",
                    "content": skill_extraction_instructions,   # strict JSON rules live here
                },
                {
                    "role": "user",
                    "content": (
                        f"Occupation Title:\n{occupation}\n\n"
                        f"Occupation Text:\n{occupation_text}"
                    ),
                },
            ],
        )

        # Should already be valid JSON
        raw_response = completion.choices[0].message.content or ""

        # Parse JSON directly
        try:
            data = json.loads(raw_response)
        except Exception as e:
            print(f"\n⚠️ JSON parse error on row {i} ({occupation}): {e}")
            print("RAW RESPONSE:", repr(raw_response))
            df_test.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Validate with Pydantic
        try:
            result = SkillList.model_validate(data)
        except Exception as e:
            print(f"\n⚠️ Schema validation error on row {i} ({occupation}): {e}")
            print("DATA:", data)
            df_test.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Save final skills
        df_test.loc[i, "llm_skills"] = json.dumps(result.skills, ensure_ascii=False)

    except Exception as e:
        print(f"\n⚠️ API error on row {i} ({occupation}): {e}")
        df_test.loc[i, "llm_skills"] = None

    time.sleep(delay)

# --- Save result ---
output_name = "10_llm_skills_groq.csv"
df_test.to_csv(output_name, index=False)
print(f"\n✅ Done! Saved results to {output_name}")


Extracting skills: 100%|██████████| 10/10 [01:48<00:00, 10.87s/job]


✅ Done! Saved results to 10_llm_skills_groq.csv


In [10]:
# --- Use the full dataframe ---
df["llm_skills"] = None   # add output column (overwrite or create)

# --- Rate limit config ---
rows_per_minute = 15
delay = 60 / rows_per_minute   # 4 seconds per request

# --- Loop over full dataset ---
for i in tqdm(range(len(df)), desc="Extracting skills", unit="job"):
    row = df.iloc[i]

    occupation = str(row.get("Element Name", "") or "").strip()
    occupation_text = str(row.get("occupation_text", "") or "").strip()

    if not occupation_text:
        df.loc[i, "llm_skills"] = None
        continue

    # ----------------------------
    # SYSTEM + USER messages
    # JSON mode ON
    # ----------------------------
    try:
        completion = client.with_options(timeout=60.0).chat.completions.create(
            model="llama-3.1-8b-instant",
            temperature=0,
            response_format={"type": "json_object"},   # 🔴 force valid JSON
            messages=[
                {
                    "role": "system",
                    "content": skill_extraction_instructions,
                },
                {
                    "role": "user",
                    "content": (
                        f"Occupation Title:\n{occupation}\n\n"
                        f"Occupation Text:\n{occupation_text}"
                    ),
                },
            ],
        )

        raw_response = completion.choices[0].message.content or ""

        # Parse JSON
        try:
            data = json.loads(raw_response)
        except Exception as e:
            print(f"\n⚠️ JSON parse error on row {i} ({occupation}): {e}")
            print("RAW RESPONSE:", repr(raw_response))
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Validate schema
        try:
            result = SkillList.model_validate(data)
        except Exception as e:
            print(f"\n⚠️ Schema validation error on row {i} ({occupation}): {e}")
            print("DATA:", data)
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Save final skills list
        df.loc[i, "llm_skills"] = json.dumps(result.skills, ensure_ascii=False)

    except Exception as e:
        print(f"\n⚠️ API error on row {i} ({occupation}): {e}")
        df.loc[i, "llm_skills"] = None

    time.sleep(delay)

    # 🔄 Autosave every 50 rows (very important for long runs)
    if (i + 1) % 50 == 0:
        df.to_csv("llm_skills_progress_groq.csv", index=False)
        print(f"💾 Autosaved at row {i+1}")

# --- Final save ---
output_name = "llm_skills_full_groq.csv"
df.to_csv(output_name, index=False)
print(f"\n✅ Done! Saved results to {output_name}")


Extracting skills:   6%|▌         | 50/894 [13:09<4:07:31, 17.60s/job]

💾 Autosaved at row 50


Extracting skills:  11%|█         | 100/894 [27:06<3:40:38, 16.67s/job]

💾 Autosaved at row 100


Extracting skills:  17%|█▋        | 150/894 [41:42<3:40:18, 17.77s/job]

...


⚠️ API error on row 892 (Refuse and Recyclable Material Collectors): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kbwxrmbmeerr1qf96ssymweq` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499977, Requested 1455. Please try again in 4m7.449599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|█████████▉| 893/894 [2:17:24<00:12, 12.96s/job]


⚠️ API error on row 893 (Tank Car, Truck, and Ship Loaders): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kbwxrmbmeerr1qf96ssymweq` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499953, Requested 1529. Please try again in 4m16.0896s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|██████████| 894/894 [2:17:28<00:00,  9.23s/job]


✅ Done! Saved results to llm_skills_full_groq.csv


In [10]:
# --- Use the full dataframe ---
df["llm_skills"] = None   # add output column (overwrite or create)

# --- Rate limit config ---
rows_per_minute = 15
delay = 60 / rows_per_minute   # 4 seconds per request

start_row = 307   # 🔥 START HERE

# --- Loop over dataset starting from row 307 ---
for i in tqdm(range(start_row, len(df)), desc="Extracting skills", unit="job"):
    row = df.iloc[i]

    occupation = str(row.get("Element Name", "") or "").strip()
    occupation_text = str(row.get("occupation_text", "") or "").strip()

    if not occupation_text:
        df.loc[i, "llm_skills"] = None
        continue

    # ----------------------------
    # SYSTEM + USER messages
    # JSON mode ON
    # ----------------------------
    try:
        completion = client.with_options(timeout=60.0).chat.completions.create(
            model="llama-3.1-8b-instant",
            temperature=0,
            response_format={"type": "json_object"},   # 🔴 force valid JSON
            messages=[
                {
                    "role": "system",
                    "content": skill_extraction_instructions,
                },
                {
                    "role": "user",
                    "content": (
                        f"Occupation Title:\n{occupation}\n\n"
                        f"Occupation Text:\n{occupation_text}"
                    ),
                },
            ],
        )

        raw_response = completion.choices[0].message.content or ""

        # Parse JSON
        try:
            data = json.loads(raw_response)
        except Exception as e:
            print(f"\n⚠️ JSON parse error on row {i} ({occupation}): {e}")
            print("RAW RESPONSE:", repr(raw_response))
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Validate schema
        try:
            result = SkillList.model_validate(data)
        except Exception as e:
            print(f"\n⚠️ Schema validation error on row {i} ({occupation}): {e}")
            print("DATA:", data)
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Save final skills list
        df.loc[i, "llm_skills"] = json.dumps(result.skills, ensure_ascii=False)

    except Exception as e:
        print(f"\n⚠️ API error on row {i} ({occupation}): {e}")
        df.loc[i, "llm_skills"] = None

    time.sleep(delay)

    # 🔄 Autosave every 50 rows
    if (i + 1) % 50 == 0:
        df.to_csv("llm_skills_progress_groq.csv", index=False)
        print(f"💾 Autosaved at row {i+1}")

# --- Final save ---
output_name = "llm_skills_full_groq.csv"
df.to_csv(output_name, index=False)
print(f"\n✅ Done! Saved results to {output_name}")


Extracting skills:   7%|▋         | 43/587 [11:24<2:35:42, 17.17s/job]

💾 Autosaved at row 350


Extracting skills:  16%|█▌        | 93/587 [25:41<2:19:56, 17.00s/job]

💾 Autosaved at row 400


Extracting skills:  24%|██▍       | 143/587 [39:55<2:06:13, 17.06s/job]

...


⚠️ API error on row 892 (Refuse and Recyclable Material Collectors): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kc01x7jbfx4twfc5hk4dtvqn` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499977, Requested 1455. Please try again in 4m7.449599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|█████████▉| 586/587 [1:50:11<00:12, 12.96s/job]


⚠️ API error on row 893 (Tank Car, Truck, and Ship Loaders): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kc01x7jbfx4twfc5hk4dtvqn` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499954, Requested 1529. Please try again in 4m16.2624s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|██████████| 587/587 [1:50:15<00:00, 11.27s/job]



✅ Done! Saved results to llm_skills_full_groq.csv


In [10]:
# --- Use the full dataframe ---
df["llm_skills"] = None   # add output column (overwrite or create)

# --- Rate limit config ---
rows_per_minute = 15
delay = 60 / rows_per_minute   # 4 seconds per request

start_row = 623   # 🔥 START HERE

# --- Loop over dataset starting from row 307 ---
for i in tqdm(range(start_row, len(df)), desc="Extracting skills", unit="job"):
    row = df.iloc[i]

    occupation = str(row.get("Element Name", "") or "").strip()
    occupation_text = str(row.get("occupation_text", "") or "").strip()

    if not occupation_text:
        df.loc[i, "llm_skills"] = None
        continue

    # ----------------------------
    # SYSTEM + USER messages
    # JSON mode ON
    # ----------------------------
    try:
        completion = client.with_options(timeout=60.0).chat.completions.create(
            model="llama-3.1-8b-instant",
            temperature=0,
            response_format={"type": "json_object"},   # 🔴 force valid JSON
            messages=[
                {
                    "role": "system",
                    "content": skill_extraction_instructions,
                },
                {
                    "role": "user",
                    "content": (
                        f"Occupation Title:\n{occupation}\n\n"
                        f"Occupation Text:\n{occupation_text}"
                    ),
                },
            ],
        )

        raw_response = completion.choices[0].message.content or ""

        # Parse JSON
        try:
            data = json.loads(raw_response)
        except Exception as e:
            print(f"\n⚠️ JSON parse error on row {i} ({occupation}): {e}")
            print("RAW RESPONSE:", repr(raw_response))
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Validate schema
        try:
            result = SkillList.model_validate(data)
        except Exception as e:
            print(f"\n⚠️ Schema validation error on row {i} ({occupation}): {e}")
            print("DATA:", data)
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Save final skills list
        df.loc[i, "llm_skills"] = json.dumps(result.skills, ensure_ascii=False)

    except Exception as e:
        print(f"\n⚠️ API error on row {i} ({occupation}): {e}")
        df.loc[i, "llm_skills"] = None

    time.sleep(delay)

    # 🔄 Autosave every 50 rows
    if (i + 1) % 50 == 0:
        df.to_csv("llm_skills_progress_groq.csv", index=False)
        print(f"💾 Autosaved at row {i+1}")

# --- Final save ---
output_name = "llm_skills_full_groq.csv"
df.to_csv(output_name, index=False)
print(f"\n✅ Done! Saved results to {output_name}")


Extracting skills:  10%|▉         | 27/271 [06:41<1:10:03, 17.23s/job]

💾 Autosaved at row 650


Extracting skills:  28%|██▊       | 77/271 [20:34<52:15, 16.16s/job]  

💾 Autosaved at row 700


Extracting skills:  47%|████▋     | 127/271 [34:47<51:44, 21.56s/job]

...


⚠️ API error on row 892 (Refuse and Recyclable Material Collectors): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kc15w8ppe1h9x74yfxagb77s` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499977, Requested 1455. Please try again in 4m7.449599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|█████████▉| 270/271 [1:03:32<00:12, 12.97s/job]


⚠️ API error on row 893 (Tank Car, Truck, and Ship Loaders): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01kc15w8ppe1h9x74yfxagb77s` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499954, Requested 1529. Please try again in 4m16.2624s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Extracting skills: 100%|██████████| 271/271 [1:03:36<00:00, 14.08s/job]


✅ Done! Saved results to llm_skills_full_groq.csv


In [10]:
# --- Use the full dataframe ---
df["llm_skills"] = None   # add output column (overwrite or create)

# --- Rate limit config ---
rows_per_minute = 15
delay = 60 / rows_per_minute   # 4 seconds per request

start_row = 835   # 🔥 START HERE

# --- Loop over dataset starting from row 307 ---
for i in tqdm(range(start_row, len(df)), desc="Extracting skills", unit="job"):
    row = df.iloc[i]

    occupation = str(row.get("Element Name", "") or "").strip()
    occupation_text = str(row.get("occupation_text", "") or "").strip()

    if not occupation_text:
        df.loc[i, "llm_skills"] = None
        continue

    # ----------------------------
    # SYSTEM + USER messages
    # JSON mode ON
    # ----------------------------
    try:
        completion = client.with_options(timeout=60.0).chat.completions.create(
            model="llama-3.1-8b-instant",
            temperature=0,
            response_format={"type": "json_object"},   # 🔴 force valid JSON
            messages=[
                {
                    "role": "system",
                    "content": skill_extraction_instructions,
                },
                {
                    "role": "user",
                    "content": (
                        f"Occupation Title:\n{occupation}\n\n"
                        f"Occupation Text:\n{occupation_text}"
                    ),
                },
            ],
        )

        raw_response = completion.choices[0].message.content or ""

        # Parse JSON
        try:
            data = json.loads(raw_response)
        except Exception as e:
            print(f"\n⚠️ JSON parse error on row {i} ({occupation}): {e}")
            print("RAW RESPONSE:", repr(raw_response))
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Validate schema
        try:
            result = SkillList.model_validate(data)
        except Exception as e:
            print(f"\n⚠️ Schema validation error on row {i} ({occupation}): {e}")
            print("DATA:", data)
            df.loc[i, "llm_skills"] = None
            time.sleep(delay)
            continue

        # Save final skills list
        df.loc[i, "llm_skills"] = json.dumps(result.skills, ensure_ascii=False)

    except Exception as e:
        print(f"\n⚠️ API error on row {i} ({occupation}): {e}")
        df.loc[i, "llm_skills"] = None

    time.sleep(delay)

    # 🔄 Autosave every 50 rows
    if (i + 1) % 50 == 0:
        df.to_csv("llm_skills_progress_groq.csv", index=False)
        print(f"💾 Autosaved at row {i+1}")

# --- Final save ---
output_name = "llm_skills_full_groq_last.csv"
df.to_csv(output_name, index=False)
print(f"\n✅ Done! Saved results to {output_name}")


Extracting skills:  25%|██▌       | 15/59 [02:52<11:39, 15.90s/job]

💾 Autosaved at row 850


Extracting skills: 100%|██████████| 59/59 [14:37<00:00, 14.87s/job]


✅ Done! Saved results to llm_skills_full_groq_last.csv
